In [1]:
import os
os.environ['PKG_CONFIG_PATH'] = '/ocean/projects/asc170022p/mtragoza/mambaforge/envs/4DCT/lib/pkgconfig'

In [2]:
import numpy as np
import fenics as fe
from fenics import grad, dot, dx, ds
import matplotlib.pyplot as plt

--------------------------------------------------------------------------

  Local host:   v015
  Local device: mlx5_1
--------------------------------------------------------------------------


In [3]:
# mesh discretization
n_nodes = 81
n_elements = n_nodes - 1
mesh = fe.UnitSquareMesh(n_elements, n_elements)
V = fe.FunctionSpace(mesh, 'Lagrange', 1)
mesh_spacing = 1 / n_elements
mesh_spacing

0.0125

In [4]:
# initial and Dirichlet boundary condition
ic_expression = fe.Constant(0)
u0 = fe.interpolate(ic_expression, V) # u_{i-2}
u1 = fe.interpolate(ic_expression, V) # u_{i-1}

def boundary(x, on_boundary):
    return on_boundary and fe.near(x[1], 0, 1e-14)

omega = 50 # Hz
bc_expression = fe.Expression(f'sin(2 * 3.1415 * omega * t)', degree=2, t=0, omega=omega)
bc = fe.DirichletBC(V, bc_expression, boundary)

# source term and Neumann boundary condition
f = fe.Expression('0', degree=0)
g = fe.Expression('0', degree=1)

In [5]:
# physical parameters

# specify wave speed in meters per second (from physical constants)
rho = 1000 # kg / m^3
mu0 = 4000 # Pa = N / m^2 = (kg m/s^2) / m^2 = kg / (m s^2)
mu1 = 16000 # Pa
c = np.sqrt(mu0 / rho) # (kg / (m s^2)) / (kg / m^3) = m^3 / (m s^2) = (m / s)^2
side_length = 0.2 # meters

# dimensionalize the wave speed
c /= side_length # side lengths per second

dt = 1e-4 # seconds

# check CFL condition
cfl_number = c * dt / mesh_spacing
print(cfl_number)
assert cfl_number <= 1, f'CFL condition violated ({c} x {dt} / {mesh_spacing} <= 1)'

total_time = 0.8 # seconds
n_steps = np.ceil(total_time / dt).astype(int)
n_steps

0.08


8000

# FEM derivation for linear elasticity

## Part 1: Derivation from first principles

First equation of motion:

\begin{align*}
    \nabla \cdot \sigma + \rho \mathbf{f} &= \rho \mathbf{u}'' \\
\end{align*}

Linear isotropic elasticity:

\begin{align*}
    \sigma &= \mathsf{E} \epsilon \\
    \sigma &= 2 \mu \epsilon + \lambda \operatorname{tr} (\epsilon) \mathbf{I} \\
\end{align*}

Kelvin-Voigt model of viscoelasticity:

\begin{align*}
    \sigma &= \mathsf{E} \epsilon + \eta \epsilon'\\
\end{align*}

Assume infinitesimal strain:

\begin{align*}
    \epsilon &= \frac{1}{2} \left( \nabla \mathbf{u} + \nabla \mathbf{u}^\top \right)
\end{align*}

Substitute:

\begin{align*}
    \nabla \cdot \left[ \mu \left(\nabla \mathbf{u} + \nabla \mathbf{u}^\top \right) + \lambda (\nabla \cdot \mathbf{u}) \mathbf{I} + \eta \epsilon' \right] + \rho \mathbf{f} &= \rho \mathbf{u}''
\end{align*}

Assume locally homogeneous (Navier-Cauchy equation):

\begin{align*}
    \mu \nabla^2 \mathbf{u} + (\lambda + \mu) \nabla (\nabla \cdot \mathbf{u}) + \nabla \cdot (\eta \epsilon') + \rho \mathbf{f} &= \rho \mathbf{u}''
\end{align*}

Assume incompressible:

\begin{align*}
    \mu \nabla^2 \mathbf{u} + \rho k \mathbf{u}' + \rho \mathbf{f} &= \rho \mathbf{u}''
\end{align*}

Define wave speed ($c = \sqrt{\mu / \rho}$):

\begin{align*}
    c^2 \nabla^2 \mathbf{u} + k \mathbf{u}' + \mathbf{f} &= \mathbf{u}''
\end{align*}


## Part 2: Time-stepping FEM formulation

Time discretization:

\begin{align*}
    c^2 \nabla^2 \mathbf{u} + \mathbf{f} &= \frac{\mathbf{u} - 2 \mathbf{u}_1 + \mathbf{u}_0}{\Delta t^2} \\
    c^2 \Delta t^2 \nabla^2 \mathbf{u} + \Delta t^2 \mathbf{f} &= \mathbf{u} - 2 \mathbf{u}_1 + \mathbf{u}_0 \\
    c^2 \Delta t^2 \nabla^2 \mathbf{u} - \mathbf{u} &= -2 \mathbf{u}_1 + \mathbf{u}_0 - \Delta t^2 \mathbf{f} \\
    (1 - c^2 \Delta t^2) \mathbf{u} &= 2 \mathbf{u}_1 - \mathbf{u}_0 + \Delta t^2 \mathbf{f}
\end{align*}

Spatial discretization:

\begin{align*}
    \int (1 - c^2 \Delta t^2) \mathbf{u} \cdot \mathbf{v} dx &= \int \left(2 \mathbf{u}_1 - \mathbf{u}_0 + \Delta t^2 \mathbf{f}\right) \cdot \mathbf{v} dx \\
    \int (1 - c^2 \Delta t^2) \mathbf{u} \cdot \mathbf{v} dx &= \int \left(2 \mathbf{u}_1 - \mathbf{u}_0 + \Delta t^2 \mathbf{f}\right) \cdot \mathbf{v} dx \\
\end{align*}

In [6]:
# variational form
u = fe.TrialFunction(V)
v = fe.TestFunction(V)
a = (
    u * v * dx + 
    (c * dt)**2 * dot(grad(u), grad(v)) * dx
)
L = (
    (2 * u1 - u0 + dt**2 * f) * v * dx + 
    (g * v) * ds
)

In [7]:
# damped inhomogeneous version

class AbsorbingBoundary(fe.UserExpression):
    def value_shape(self):
        return ()
    def eval(self, values, x):
        if x[0] < 0.05 or x[0] > 0.95 or x[1] > 0.95:
            values[0] = -1000
        else:
            values[0] = -10

k = fe.interpolate(AbsorbingBoundary(), V)

class InhomogeneousMaterial(fe.UserExpression):
    def value_shape(self):
        return ()
    def eval(self, values, x):
        dist = np.sqrt((x[0] - 0.5)**2 + (x[1] - 0.5)**2)
        if dist < 0.25: # target
            values[0] = np.sqrt(mu1 / rho) / side_length
        else: # background
            values[0] = np.sqrt(mu0 / rho) / side_length

c = fe.interpolate(InhomogeneousMaterial(), V)

u = fe.TrialFunction(V)
v = fe.TestFunction(V)
a = (
    (2-k*dt) * u*v + 
    2*(c*dt)**2 * dot(grad(u),grad(v))
) * dx
L = (
    (4*u1 - (2+k*dt)*u0 + 2*dt**2 * f) * v * dx +
    (g*v) * ds
)

In [8]:
# get mapping from dofs to array indices
xyz = V.tabulate_dof_coordinates()
x = xyz[:,0]
y = xyz[:,1]
print(xyz.shape)

inds = np.argsort([n_nodes * x + y for x,y in xyz])
xyz[inds]

(6561, 2)


array([[0.    , 0.    ],
       [0.    , 0.0125],
       [0.    , 0.025 ],
       ...,
       [1.    , 0.975 ],
       [1.    , 0.9875],
       [1.    , 1.    ]])

In [9]:
import tqdm

u0.interpolate(ic_expression)
u1.interpolate(ic_expression)

u = fe.Function(V)
array = np.zeros((n_steps + 1, n_nodes, n_nodes))

t = 0
for i in tqdm.tqdm(range(n_steps)):
    t += dt
    bc_expression.t = t
    fe.solve(a == L, u, [bc])
    u0.assign(u1)
    u1.assign(u)

    values = u.vector().get_local()
    array[i+1,...] = values[inds].reshape((n_nodes, n_nodes))


  0%|          | 0/8000 [00:00<?, ?it/s]

Solving linear variational problem.


  0%|          | 7/8000 [00:01<22:14,  5.99it/s]  

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.


  0%|          | 13/8000 [00:01<10:48, 12.31it/s]

...

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.


100%|█████████▉| 7996/8000 [04:46<00:00, 28.45it/s]

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.


100%|██████████| 8000/8000 [04:46<00:00, 27.91it/s]

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.


In [10]:
import xarray as xr
import hvplot.xarray

array = xr.DataArray(
    array,
    dims=['t', 'x', 'y'],
    coords={
        't': np.arange(0, n_steps + 1) * dt,
        'x': np.linspace(0, 1, n_nodes),
        'y': np.linspace(0, 1, n_nodes)
    }
)
array.hvplot(
    groupby=['t'],
    x='x',
    y='y',
    data_aspect=1,
    clim=(-2, 2),
    cmap='seismic',
    widget_type='scrubber',
    widget_location='bottom'
)

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom', widget_type='scrubber')
    [1] WidgetBox(align=('center', 'end'))
        [0] Player(end=8000, width=550)

In [11]:
array.hvplot(groupby=['t', 'x'], x='y', ylim=[-2, 2], widget_location='bottom')

Column
    [0] HoloViews(DynamicMap, height=300, sizing_mode='fixed', widget_location='bottom', width=700)
    [1] WidgetBox(align=('center', 'end'))
        [0] DiscreteSlider(margin=(20, 20, 5, 20), name='t', options=OrderedDict([('0', ...]), value=0.0, width=250)
        [1] DiscreteSlider(margin=(5, 20, 20, 20), name='x', options=OrderedDict([('0', ...]), value=0.0, width=250)

In [12]:
array

<xarray.DataArray (t: 8001, x: 81, y: 81)>
array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 3.14098330e-02, -2.53910869e-03, -1.15101909e-05, ...,
          7.02249070e-65,  8.15696123e-66, -1.14256793e-65],
        [ 3.14098330e-02, -8.65559226e-03,  1.46037905e-03, ...,
          3.24287782e-64, -7.68342747e-65,  1.83010236e-65],
        [ 3.14098330e-02, -7.80422922e-03,  2.17690539e-03, ...,
         -1.00067765e-63,  7.00480340e-65,  2.32445144e-65],
...
        [-3.88175322e-02, -4.19096549e-01, -4.87515551e-01, ...,
          1.34238818e-02,  2.30799550e-02,  2.65554283e-02],
        [-3.88175322e-02, -4.10301972e-01, -4.53476452e-01, ...,
          2.20321061e-02,  2.64788969e-02,  2.80301369e-02],
        [-3.88175322e-02, -4.13338602e-01, -4.48202704e-01, ...,
          2.52749559e-02,  2.78046824e-02,  2.86280438e-02]],

       [[-7.41221933e-03, -3.65172912e-01, -4.07336398e-01, ...,
          2.33430314e-02,  2.53865859e-02,  2.58451886e-02],
        [-7.41221933e-03, -3.82292464e-01, -4.33195650e-01, ...,
          2.12256296e-02,  2.47466008e-02,  2.56840499e-02],
        [-7.41221933e-03, -4.03217541e-01, -4.80432157e-01, ...,
          1.40412568e-02,  2.22322949e-02,  2.46662557e-02],
        ...,
        [-7.41221933e-03, -4.01341159e-01, -4.81804438e-01, ...,
          1.28388518e-02,  2.27554049e-02,  2.63344053e-02],
        [-7.41221933e-03, -3.93588885e-01, -4.49344084e-01, ...,
          2.16523984e-02,  2.64090820e-02,  2.80808061e-02],
        [-7.41221933e-03, -3.96742434e-01, -4.44455283e-01, ...,
          2.49732013e-02,  2.78326376e-02,  2.88070055e-02]]])
Coordinates:
  * t        (t) float64 0.0 0.0001 0.0002 0.0003 ... 0.7997 0.7998 0.7999 0.8
  * x        (x) float64 0.0 0.0125 0.025 0.0375 ... 0.9625 0.975 0.9875 1.0
  * y        (y) float64 0.0 0.0125 0.025 0.0375 ... 0.9625 0.975 0.9875 1.0

In [ ]:
%%time
import os
import subprocess

def create_video(video_name, array):
    
    print('Creating video frames...')
    os.makedirs(video_name, exist_ok=True)
    for i, frame in enumerate(array):
        frame_file = video_name + f'/frame{i:04d}.png'
        plt.imshow(frame.T, vmin=-2, vmax=2, cmap='seismic')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.savefig(frame_file)
        print(frame_file)
        
    print('Creating video...')
    os.chdir(video_name)
    subprocess.call([
        'ffmpeg', '-framerate', '30', '-i', 'file%04d.png', '-r', '30', '-pix_fmt', 'yuv420p', f'{video_name}.mp4'
    ])

create_video('my_video4', array.values)

Creating video frames...
my_video4/frame0000.png
my_video4/frame0001.png
my_video4/frame0002.png
my_video4/frame0003.png
my_video4/frame0004.png
my_video4/frame0005.png
my_video4/frame0006.png
my_video4/frame0007.png
my_video4/frame0008.png
my_video4/frame0009.png
my_video4/frame0010.png
my_video4/frame0011.png
my_video4/frame0012.png
my_video4/frame0013.png
my_video4/frame0014.png
my_video4/frame0015.png
my_video4/frame0016.png
my_video4/frame0017.png
my_video4/frame0018.png
my_video4/frame0019.png
my_video4/frame0020.png
my_video4/frame0021.png
my_video4/frame0022.png
my_video4/frame0023.png
my_video4/frame0024.png
my_video4/frame0025.png
my_video4/frame0026.png
my_video4/frame0027.png
my_video4/frame0028.png
my_video4/frame0029.png
my_video4/frame0030.png
my_video4/frame0031.png
my_video4/frame0032.png
my_video4/frame0033.png
my_video4/frame0034.png
my_video4/frame0035.png
my_video4/frame0036.png
my_video4/frame0037.png
my_video4/frame0038.png
my_video4/frame0039.png
my_video4/frame

my_video4/frame0341.png
my_video4/frame0342.png
my_video4/frame0343.png
my_video4/frame0344.png
my_video4/frame0345.png
my_video4/frame0346.png
my_video4/frame0347.png
my_video4/frame0348.png
my_video4/frame0349.png
my_video4/frame0350.png
my_video4/frame0351.png
my_video4/frame0352.png
my_video4/frame0353.png
my_video4/frame0354.png
my_video4/frame0355.png
my_video4/frame0356.png
my_video4/frame0357.png
my_video4/frame0358.png
my_video4/frame0359.png
my_video4/frame0360.png
my_video4/frame0361.png
my_video4/frame0362.png
my_video4/frame0363.png
my_video4/frame0364.png
my_video4/frame0365.png
my_video4/frame0366.png
my_video4/frame0367.png
my_video4/frame0368.png
my_video4/frame0369.png
my_video4/frame0370.png
my_video4/frame0371.png
my_video4/frame0372.png
my_video4/frame0373.png
my_video4/frame0374.png
my_video4/frame0375.png
my_video4/frame0376.png
my_video4/frame0377.png
my_video4/frame0378.png
my_video4/frame0379.png
my_video4/frame0380.png
my_video4/frame0381.png
my_video4/frame0

my_video4/frame0683.png
my_video4/frame0684.png
my_video4/frame0685.png
my_video4/frame0686.png
my_video4/frame0687.png
my_video4/frame0688.png
my_video4/frame0689.png
my_video4/frame0690.png
my_video4/frame0691.png
my_video4/frame0692.png
my_video4/frame0693.png
my_video4/frame0694.png
my_video4/frame0695.png
my_video4/frame0696.png
my_video4/frame0697.png
my_video4/frame0698.png
my_video4/frame0699.png
my_video4/frame0700.png
my_video4/frame0701.png
my_video4/frame0702.png
my_video4/frame0703.png
my_video4/frame0704.png
my_video4/frame0705.png
my_video4/frame0706.png
my_video4/frame0707.png
my_video4/frame0708.png
my_video4/frame0709.png
my_video4/frame0710.png
my_video4/frame0711.png
my_video4/frame0712.png
my_video4/frame0713.png
my_video4/frame0714.png
my_video4/frame0715.png
my_video4/frame0716.png
my_video4/frame0717.png
my_video4/frame0718.png
my_video4/frame0719.png
my_video4/frame0720.png
my_video4/frame0721.png
my_video4/frame0722.png
my_video4/frame0723.png
my_video4/frame0

my_video4/frame1025.png
my_video4/frame1026.png
my_video4/frame1027.png
my_video4/frame1028.png
my_video4/frame1029.png
my_video4/frame1030.png
my_video4/frame1031.png
my_video4/frame1032.png
my_video4/frame1033.png
my_video4/frame1034.png
my_video4/frame1035.png
my_video4/frame1036.png
my_video4/frame1037.png
my_video4/frame1038.png
my_video4/frame1039.png
my_video4/frame1040.png
my_video4/frame1041.png
my_video4/frame1042.png
my_video4/frame1043.png
my_video4/frame1044.png
my_video4/frame1045.png
my_video4/frame1046.png
my_video4/frame1047.png
my_video4/frame1048.png
my_video4/frame1049.png
my_video4/frame1050.png
my_video4/frame1051.png
my_video4/frame1052.png
my_video4/frame1053.png
my_video4/frame1054.png
my_video4/frame1055.png
my_video4/frame1056.png
my_video4/frame1057.png
my_video4/frame1058.png
my_video4/frame1059.png
my_video4/frame1060.png
my_video4/frame1061.png
my_video4/frame1062.png
my_video4/frame1063.png
my_video4/frame1064.png
my_video4/frame1065.png
my_video4/frame1

my_video4/frame1367.png
my_video4/frame1368.png
my_video4/frame1369.png
my_video4/frame1370.png
my_video4/frame1371.png
my_video4/frame1372.png
my_video4/frame1373.png
my_video4/frame1374.png
my_video4/frame1375.png
my_video4/frame1376.png
my_video4/frame1377.png
my_video4/frame1378.png
my_video4/frame1379.png
my_video4/frame1380.png
my_video4/frame1381.png
my_video4/frame1382.png
my_video4/frame1383.png
my_video4/frame1384.png
my_video4/frame1385.png
my_video4/frame1386.png
my_video4/frame1387.png
my_video4/frame1388.png
my_video4/frame1389.png
my_video4/frame1390.png
my_video4/frame1391.png
my_video4/frame1392.png
my_video4/frame1393.png
my_video4/frame1394.png
my_video4/frame1395.png
my_video4/frame1396.png
my_video4/frame1397.png
my_video4/frame1398.png
my_video4/frame1399.png
my_video4/frame1400.png
my_video4/frame1401.png
my_video4/frame1402.png
my_video4/frame1403.png
my_video4/frame1404.png
my_video4/frame1405.png
my_video4/frame1406.png
my_video4/frame1407.png
my_video4/frame1

...

my_video4/frame2393.png
my_video4/frame2394.png
my_video4/frame2395.png
my_video4/frame2396.png
my_video4/frame2397.png
my_video4/frame2398.png
my_video4/frame2399.png
my_video4/frame2400.png
my_video4/frame2401.png
my_video4/frame2402.png
my_video4/frame2403.png
my_video4/frame2404.png
my_video4/frame2405.png
my_video4/frame2406.png
my_video4/frame2407.png
my_video4/frame2408.png
my_video4/frame2409.png
my_video4/frame2410.png
my_video4/frame2411.png
my_video4/frame2412.png
my_video4/frame2413.png
my_video4/frame2414.png
my_video4/frame2415.png
my_video4/frame2416.png
my_video4/frame2417.png
my_video4/frame2418.png
my_video4/frame2419.png
my_video4/frame2420.png
my_video4/frame2421.png
my_video4/frame2422.png
my_video4/frame2423.png
my_video4/frame2424.png
my_video4/frame2425.png
my_video4/frame2426.png
my_video4/frame2427.png
my_video4/frame2428.png
my_video4/frame2429.png
my_video4/frame2430.png
my_video4/frame2431.png
my_video4/frame2432.png
my_video4/frame2433.png
my_video4/frame2

my_video4/frame2735.png
my_video4/frame2736.png
my_video4/frame2737.png
my_video4/frame2738.png
my_video4/frame2739.png
my_video4/frame2740.png
my_video4/frame2741.png
my_video4/frame2742.png
my_video4/frame2743.png
my_video4/frame2744.png
my_video4/frame2745.png
my_video4/frame2746.png
my_video4/frame2747.png
my_video4/frame2748.png
my_video4/frame2749.png
my_video4/frame2750.png
my_video4/frame2751.png
my_video4/frame2752.png
my_video4/frame2753.png
my_video4/frame2754.png
my_video4/frame2755.png
my_video4/frame2756.png
my_video4/frame2757.png
my_video4/frame2758.png
my_video4/frame2759.png
my_video4/frame2760.png
my_video4/frame2761.png
my_video4/frame2762.png
my_video4/frame2763.png
my_video4/frame2764.png
my_video4/frame2765.png
my_video4/frame2766.png
my_video4/frame2767.png
my_video4/frame2768.png
my_video4/frame2769.png
my_video4/frame2770.png
my_video4/frame2771.png
my_video4/frame2772.png
my_video4/frame2773.png
my_video4/frame2774.png
my_video4/frame2775.png
my_video4/frame2

my_video4/frame3077.png
my_video4/frame3078.png
my_video4/frame3079.png
my_video4/frame3080.png
my_video4/frame3081.png
my_video4/frame3082.png
my_video4/frame3083.png
my_video4/frame3084.png
my_video4/frame3085.png
my_video4/frame3086.png
my_video4/frame3087.png
my_video4/frame3088.png
my_video4/frame3089.png
my_video4/frame3090.png
my_video4/frame3091.png
my_video4/frame3092.png
my_video4/frame3093.png
my_video4/frame3094.png
my_video4/frame3095.png
my_video4/frame3096.png
my_video4/frame3097.png
my_video4/frame3098.png
my_video4/frame3099.png
my_video4/frame3100.png
my_video4/frame3101.png
my_video4/frame3102.png
my_video4/frame3103.png
my_video4/frame3104.png
my_video4/frame3105.png
my_video4/frame3106.png
my_video4/frame3107.png
my_video4/frame3108.png
my_video4/frame3109.png
my_video4/frame3110.png
my_video4/frame3111.png
my_video4/frame3112.png
my_video4/frame3113.png
my_video4/frame3114.png
my_video4/frame3115.png
my_video4/frame3116.png
my_video4/frame3117.png
my_video4/frame3

my_video4/frame3419.png
my_video4/frame3420.png
my_video4/frame3421.png
my_video4/frame3422.png
my_video4/frame3423.png
my_video4/frame3424.png
my_video4/frame3425.png
my_video4/frame3426.png
my_video4/frame3427.png
my_video4/frame3428.png
my_video4/frame3429.png
my_video4/frame3430.png
my_video4/frame3431.png
my_video4/frame3432.png
my_video4/frame3433.png
my_video4/frame3434.png
my_video4/frame3435.png
my_video4/frame3436.png
my_video4/frame3437.png
my_video4/frame3438.png
my_video4/frame3439.png
my_video4/frame3440.png
my_video4/frame3441.png
my_video4/frame3442.png
my_video4/frame3443.png
my_video4/frame3444.png
my_video4/frame3445.png
my_video4/frame3446.png
my_video4/frame3447.png
my_video4/frame3448.png
my_video4/frame3449.png
my_video4/frame3450.png
my_video4/frame3451.png
my_video4/frame3452.png
my_video4/frame3453.png
my_video4/frame3454.png
my_video4/frame3455.png
my_video4/frame3456.png
my_video4/frame3457.png
my_video4/frame3458.png
my_video4/frame3459.png
my_video4/frame3

my_video4/frame3761.png
my_video4/frame3762.png
my_video4/frame3763.png
my_video4/frame3764.png
my_video4/frame3765.png
my_video4/frame3766.png
my_video4/frame3767.png
my_video4/frame3768.png
my_video4/frame3769.png
my_video4/frame3770.png
my_video4/frame3771.png
my_video4/frame3772.png
my_video4/frame3773.png
my_video4/frame3774.png
my_video4/frame3775.png
my_video4/frame3776.png
my_video4/frame3777.png
my_video4/frame3778.png
my_video4/frame3779.png
my_video4/frame3780.png
my_video4/frame3781.png
my_video4/frame3782.png
my_video4/frame3783.png
my_video4/frame3784.png
my_video4/frame3785.png
my_video4/frame3786.png
my_video4/frame3787.png
my_video4/frame3788.png
my_video4/frame3789.png
my_video4/frame3790.png
my_video4/frame3791.png
my_video4/frame3792.png
my_video4/frame3793.png
my_video4/frame3794.png
my_video4/frame3795.png
my_video4/frame3796.png
my_video4/frame3797.png
my_video4/frame3798.png
my_video4/frame3799.png
my_video4/frame3800.png
my_video4/frame3801.png
my_video4/frame3